## Caffe2 Concepts
### Blobs and Workspace,Tensor

In [2]:
from caffe2.python import workspace,model_helper
import numpy as np
x = np.random.rand(3,2,3)
print x
workspace.ResetWorkspace()
workspace.FeedBlob('my_x', x)
print workspace.FetchBlob('my_x')
print workspace.FetchBlob('my_x').shape

[[[0.46941123 0.44155008 0.2736049 ]
  [0.21484632 0.9609905  0.29437779]]

 [[0.01855038 0.26009939 0.38142868]
  [0.93254492 0.50404126 0.29565106]]

 [[0.32619875 0.31005314 0.33727066]
  [0.71978962 0.3776309  0.40083127]]]
[[[0.46941123 0.44155008 0.2736049 ]
  [0.21484632 0.9609905  0.29437779]]

 [[0.01855038 0.26009939 0.38142868]
  [0.93254492 0.50404126 0.29565106]]

 [[0.32619875 0.31005314 0.33727066]
  [0.71978962 0.3776309  0.40083127]]]
(3, 2, 3)


### Nets and Operators

In [5]:
# Create the input data
data = np.random.rand(16,100).astype(np.float32)
label = (np.random.rand(16)*10).astype(np.int32)
workspace.FeedBlob('data', data)
workspace.FeedBlob('label', label)

True

In [20]:
# Create model using a model helper,will create two net:init_net for param initiate, exec_net for run
m = model_helper.ModelHelper(name='my first net')  # if run this code block for mutiple times, the model name may change
# Generate weight and bias
weight = m.param_init_net.XavierFill([], 'fc_w', shape=[10,100])
bias = m.param_init_net.ConstantFill([], 'fc_b', shape=[10,])
# Build FC,Sigmoid and SoftmaxWithLoss
fc_1 = m.net.FC(['data','fc_w','fc_b'], 'fc_1')
pred = m.net.Sigmoid(fc_1, 'pred')
softmax, loss = m.net.SoftmaxWithLoss([pred, 'label'], ['softmax', 'loss'])
# Add backword pass
m.AddGradientOperators([loss])
print m.net.Proto()

name: "my first net"
op {
  input: "data"
  input: "fc_w"
  input: "fc_b"
  output: "fc_1"
  name: ""
  type: "FC"
}
op {
  input: "fc_1"
  output: "pred"
  name: ""
  type: "Sigmoid"
}
op {
  input: "pred"
  input: "label"
  output: "softmax"
  output: "loss"
  name: ""
  type: "SoftmaxWithLoss"
}
op {
  input: "loss"
  output: "loss_autogen_grad"
  name: ""
  type: "ConstantFill"
  arg {
    name: "value"
    f: 1.0
  }
}
op {
  input: "pred"
  input: "label"
  input: "softmax"
  input: "loss_autogen_grad"
  output: "pred_grad"
  name: ""
  type: "SoftmaxWithLossGradient"
  is_gradient_op: true
}
op {
  input: "pred"
  input: "pred_grad"
  output: "fc_1_grad"
  name: ""
  type: "SigmoidGradient"
  is_gradient_op: true
}
op {
  input: "data"
  input: "fc_w"
  input: "fc_1_grad"
  output: "fc_w_grad"
  output: "fc_b_grad"
  output: "data_grad"
  name: ""
  type: "FCGradient"
  is_gradient_op: true
}
external_input: "data"
external_input: "fc_w"
external_input: "fc_b"
external_input: "l

In [23]:
print m.param_init_net.Proto()

name: "ygj first net_init"
op {
  output: "fc_w"
  name: ""
  type: "XavierFill"
  arg {
    name: "shape"
    ints: 10
    ints: 100
  }
}
op {
  output: "fc_b"
  name: ""
  type: "ConstantFill"
  arg {
    name: "shape"
    ints: 10
  }
}



### Executing

In [17]:
# Run once the param initialization and create the actual training Net
workspace.RunNetOnce(m.param_init_net)
workspace.CreateNet(m.net, overwrite=True)
# Run 100 x 10 iter
for _ in range(1000):
    data = np.random.rand(16,100).astype(np.float32)
    label = (np.random.rand(16)*10).astype(np.int32)
    workspace.FeedBlob('data', data)
    workspace.FeedBlob('label', label)
    workspace.RunNet(m.name, 10)

In [18]:
# print workspace.FetchBlob('softmax')
print workspace.FetchBlob('loss')

2.269385
